In [1]:
# Python 3.6.1

# import packages
import requests
import numpy as np
import pandas as pd
from bis2 import gc2
from collections import OrderedDict 
from bokeh.io import output_notebook
from bokeh.plotting import figure, show

In [2]:
#Define database information needed for the notebook to run
#This currently requires the bis2 package, which is 
thisRun = {}
thisRun['url'] = gc2.baseURLs["sqlapi_datadistillery_bcb_beta"]

thisRun['fromSchema'] = 'nfhp.'
thisRun['fromTable'] = 'hci2015_summaries_mp'

thisRun['fromSchemaTable'] = thisRun['fromSchema']+thisRun['fromTable']

In [3]:
#Function that passes base url, table and place of interst (supplied from user or application) 
#to extract summarized information of interest from the BIS database (GC2).

def requestHci(url, fromTable, place):
    import requests
    
    queryHci = url+"?q=select place_name, scored_km, not_scored_km, verylow_km, low_km, moderate_km, high_km, veryhigh_km \
    from " + fromTable + " where source_id='" + place + "'"
    
    hciRequest = requests.get(url=queryHci, verify=True).json()
    data = hciRequest['features'][0]['properties']
    title = 'Risk to Fish Habitat Degradation in '+ str(data['place_name'] + ' Streams')
    
    #Reorganize data into dictionary that will support display using bokeh bar chart
    summary_dict = OrderedDict([('risk',0)])
    summary_dict['risk'] = OrderedDict([ ('veryLow',((float(data['verylow_km']))/(float(data['scored_km'])))*100),
('low',((float(data['low_km']))/(float(data['scored_km'])))*100),
('moderate',((float(data['moderate_km']))/(float(data['scored_km'])))*100),
('high',((float(data['high_km']))/(float(data['scored_km'])))*100),
('veryHigh',((float(data['veryhigh_km']))/(float(data['scored_km'])))*100)])
    
    #Send reorganized data into pandas dataframe
    dataDf = pd.DataFrame(summary_dict, index=summary_dict['risk'].keys()) 
    
    return dataDf, title

In [4]:
place_source_id = 'gu_state:783534a7-82aa-4a24-9eb4-3a6549f1b295'

dataDf, title = requestHci(thisRun['url'],thisRun['fromSchemaTable'], place_source_id)

dataDf

,risk
veryLow,37.223554
low,44.201776
moderate,12.652380
high,4.989542
veryHigh,0.932749


In [5]:
# Create Bokeh Bar Chart using dataframe

series = dataDf.loc[:,'risk']

p = figure(width=800, height=600, y_range=series.index.tolist())
p.background_fill_color = "white"
p.grid.grid_line_alpha=1.0
p.grid.grid_line_color = None
p.xaxis.axis_label = 'Percentage of NFHP Scored Stream Kms [%]'
p.xaxis.axis_label_text_font_size = '14pt'
p.xaxis.major_label_text_font_size = '14pt'
p.yaxis.major_label_text_font_size = '14pt'
p.yaxis.axis_label = 'Risk to Fish Habitat Degradation'
p.yaxis.axis_label_text_font_size = '14pt'

p.title.text = title
p.title.align = "center"
p.title.text_color = "orange"
p.title.text_font_size = "22px"

# bar colors
bcolors = ['violet', 'cyan', '#a1dab4','orange', 'red']

# plot order
order = ['veryLow','low','moderate','high','veryHigh']

j = 1
for k in order: #[::-1]:
    v = series[k]
#   print (k,v,j)
    p.rect(x=v/2, y=j, width=abs(v), height=0.8,color=bcolors[j-1],
    width_units="data", height_units="data")
    j += 1
    
output_notebook()
show(p)




Loading BokehJS ...